In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,  make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from sklearn.dummy import DummyClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

from sklearn.multioutput import MultiOutputClassifier

import pickle

In [3]:
unpickleFile = open('./Data/clean_people.pkl', 'rb')
people_df = pickle.load(unpickleFile, encoding='bytes')
people_df.head()

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,DRIVERS_LICENSE_CLASS,SEX,AGE,SAFETY_EQUIPMENT
0,O749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,D,M,25.0,NONE PRESENT
1,O871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,NaN,M,37.0,SAFETY BELT USED
2,O10018,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,NaN,X,NaN,USAGE UNKNOWN
3,O10038,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,9598.0,NaN,X,NaN,USAGE UNKNOWN
4,O10039,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,9600.0,NaN,X,NaN,USAGE UNKNOWN


In [4]:
unpickleFile = open('./Data/clean_vehicles.pkl', 'rb')
vehicle_df = pickle.load(unpickleFile, encoding='bytes')
vehicle_df.head()

,CRASH_RECORD_ID,VEHICLE_ID,UNIT_TYPE,NUM_PASSENGERS,MAKE,MODEL,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,MANEUVER,FIRST_CONTACT_POINT
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,796949.0,DRIVER,0.0,INFINITI,UNKNOWN,NONE,PASSENGER,PERSONAL,STRAIGHT AHEAD,FRONT
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,DRIVER,0.0,HONDA,CIVIC,UNKNOWN,PASSENGER,PERSONAL,STRAIGHT AHEAD,FRONT
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834819.0,PARKED,0.0,TOYOTA,YARIS,NONE,UNKNOWN/NA,PERSONAL,PARKED,ROOF
3,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834817.0,PARKED,0.0,GENERAL MOTORS CORPORATION (GMC),SIERRA,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,PARKED,ROOF
4,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,DRIVER,0.0,BUICK,ENCORE,NONE,PASSENGER,PERSONAL,STRAIGHT AHEAD,FRONT-RIGHT


In [6]:
unpickleFile = open('./Data/clean_crashes.pkl', 'rb')
crash_df = pickle.load(unpickleFile, encoding='bytes')
crash_df.head()

,CRASH_RECORD_ID,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_DIRECTION,BEAT_OF_OCCURRENCE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,INJURY_LEVEL
0,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,30,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,NO INJURY / DRIVE AWAY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,N,1123.0,0,6,7,0
1,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,30,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,N,1811.0,18,6,7,0
2,01aaa759c6bbefd0f584226fbd88bdc549de3ed1e46255...,40,CLOUDY/OVERCAST,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,IMPROPER LANE USAGE,NOT APPLICABLE,N,2024.0,17,6,7,0
3,0362b3f8ddb8589dfe889356efbc2ad213efe2031556f0...,30,RAIN,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,WET,NO DEFECTS,NO INJURY / DRIVE AWAY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,N,1634.0,8,6,7,0
4,03d3679ef44bb8aa0f2060cb0376f3eeb1d9dbb2197322...,30,RAIN,DAYLIGHT,PEDESTRIAN,FOUR WAY,STRAIGHT AND LEVEL,WET,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,FAILING TO YIELD RIGHT-OF-WAY,UNABLE TO DETERMINE,S,1235.0,16,6,7,1


# Goal

My goal here is to create a df that contains a row for every driver of a crash.

This row will contain unique info about the driver and vehicle they were driving. There will be duplicates of the crash condition data for each person/vehicle, as this data will apply to multiple drivers if there were multiple driven cars involved in the crash.

In [8]:
driver_vehicle = pd.DataFrame()

In [11]:
vehicle_df.drop(columns=['CRASH_RECORD_ID'], inplace=True)

In [13]:
# Create df of driver and vehicle for every driver
driver_vehicle = people_df.join(vehicle_df.set_index('VEHICLE_ID'),
                               on = 'VEHICLE_ID',
                               how = 'left')

In [14]:
driver_vehicle.head()

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,DRIVERS_LICENSE_CLASS,SEX,AGE,SAFETY_EQUIPMENT,UNIT_TYPE,NUM_PASSENGERS,MAKE,MODEL,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,MANEUVER,FIRST_CONTACT_POINT
0,O749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,D,M,25.0,NONE PRESENT,DRIVER,0.0,HONDA,CIVIC,UNKNOWN,PASSENGER,PERSONAL,STRAIGHT AHEAD,FRONT
1,O871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,NaN,M,37.0,SAFETY BELT USED,DRIVER,0.0,BUICK,ENCORE,NONE,PASSENGER,PERSONAL,STRAIGHT AHEAD,FRONT-RIGHT
2,O10018,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,NONE,PASSENGER,PERSONAL,BACKING,REAR
3,O10038,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,9598.0,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN
4,O10039,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,9600.0,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN


In [15]:
# drop unneccesary cols
driver_vehicle.drop(columns=['VEHICLE_ID', 'PERSON_ID'], inplace=True)
driver_vehicle.head()

,CRASH_RECORD_ID,DRIVERS_LICENSE_CLASS,SEX,AGE,SAFETY_EQUIPMENT,UNIT_TYPE,NUM_PASSENGERS,MAKE,MODEL,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,MANEUVER,FIRST_CONTACT_POINT
0,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,D,M,25.0,NONE PRESENT,DRIVER,0.0,HONDA,CIVIC,UNKNOWN,PASSENGER,PERSONAL,STRAIGHT AHEAD,FRONT
1,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,NaN,M,37.0,SAFETY BELT USED,DRIVER,0.0,BUICK,ENCORE,NONE,PASSENGER,PERSONAL,STRAIGHT AHEAD,FRONT-RIGHT
2,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,NONE,PASSENGER,PERSONAL,BACKING,REAR
3,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN
4,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN


In [16]:
driver_vehicle.shape

(1121892, 14)

In [17]:
dv_crash = pd.DataFrame()

In [18]:
# Create df of driver+vehicle for every driver, with crash info
dv_crash = driver_vehicle.join(crash_df.set_index('CRASH_RECORD_ID'),
                               on = 'CRASH_RECORD_ID',
                               how = 'left')

In [19]:
dv_crash.head()

,CRASH_RECORD_ID,DRIVERS_LICENSE_CLASS,SEX,AGE,SAFETY_EQUIPMENT,UNIT_TYPE,NUM_PASSENGERS,MAKE,MODEL,VEHICLE_DEFECT,...,ROAD_DEFECT,CRASH_TYPE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_DIRECTION,BEAT_OF_OCCURRENCE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,INJURY_LEVEL
0,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,D,M,25.0,NONE PRESENT,DRIVER,0.0,HONDA,CIVIC,UNKNOWN,...,NO DEFECTS,NO INJURY / DRIVE AWAY,UNABLE TO DETERMINE,NOT APPLICABLE,N,1531.0,3.0,7.0,9.0,0
1,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,NaN,M,37.0,SAFETY BELT USED,DRIVER,0.0,BUICK,ENCORE,NONE,...,NO DEFECTS,NO INJURY / DRIVE AWAY,IMPROPER OVERTAKING/PASSING,FAILING TO REDUCE SPEED TO AVOID CRASH,W,613.0,22.0,2.0,4.0,0
2,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,NONE,...,NO DEFECTS,NO INJURY / DRIVE AWAY,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,NOT APPLICABLE,W,821.0,5.0,1.0,11.0,0
3,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,UNKNOWN,...,NO DEFECTS,NO INJURY / DRIVE AWAY,UNABLE TO DETERMINE,UNABLE TO DETERMINE,N,2023.0,8.0,1.0,11.0,0
4,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,NaN,X,NaN,USAGE UNKNOWN,DRIVER,0.0,UNKNOWN,UNKNOWN,UNKNOWN,...,NO DEFECTS,NO INJURY / DRIVE AWAY,UNABLE TO DETERMINE,NOT APPLICABLE,S,223.0,10.0,1.0,11.0,0


In [20]:
# drop unneccary cols
dv_crash.drop(columns=['CRASH_RECORD_ID'], inplace=True)

In [21]:
dv_crash.shape

(1121892, 30)

# Some additional cleaning

In [22]:
dv_crash.isna().sum()

DRIVERS_LICENSE_CLASS      415017
SEX                             0
AGE                        296536
SAFETY_EQUIPMENT                0
UNIT_TYPE                   33172
NUM_PASSENGERS              33172
MAKE                        33172
MODEL                       33172
VEHICLE_DEFECT              33172
VEHICLE_TYPE                33172
VEHICLE_USE                 33172
MANEUVER                    33172
FIRST_CONTACT_POINT         33172
POSTED_SPEED_LIMIT            598
WEATHER_CONDITION             598
LIGHTING_CONDITION            598
FIRST_CRASH_TYPE              598
TRAFFICWAY_TYPE               598
ALIGNMENT                     598
ROADWAY_SURFACE_COND          598
ROAD_DEFECT                   598
CRASH_TYPE                    598
PRIM_CONTRIBUTORY_CAUSE       598
SEC_CONTRIBUTORY_CAUSE        598
STREET_DIRECTION              598
BEAT_OF_OCCURRENCE            598
CRASH_HOUR                    598
CRASH_DAY_OF_WEEK             598
CRASH_MONTH                   598
INJURY_LEVEL  

In [23]:
# Drop 598 rows that are missing speed limit, weather condition, etc
dv_crash.dropna(subset=['POSTED_SPEED_LIMIT'], inplace=True)
dv_crash.isna().sum()

DRIVERS_LICENSE_CLASS      414774
SEX                             0
AGE                        296379
SAFETY_EQUIPMENT                0
UNIT_TYPE                   33148
NUM_PASSENGERS              33148
MAKE                        33148
MODEL                       33148
VEHICLE_DEFECT              33148
VEHICLE_TYPE                33148
VEHICLE_USE                 33148
MANEUVER                    33148
FIRST_CONTACT_POINT         33148
POSTED_SPEED_LIMIT              0
WEATHER_CONDITION               0
LIGHTING_CONDITION              0
FIRST_CRASH_TYPE                0
TRAFFICWAY_TYPE                 0
ALIGNMENT                       0
ROADWAY_SURFACE_COND            0
ROAD_DEFECT                     0
CRASH_TYPE                      0
PRIM_CONTRIBUTORY_CAUSE         0
SEC_CONTRIBUTORY_CAUSE          0
STREET_DIRECTION                0
BEAT_OF_OCCURRENCE              0
CRASH_HOUR                      0
CRASH_DAY_OF_WEEK               0
CRASH_MONTH                     0
INJURY_LEVEL  

In [24]:
dv_crash.shape

(1121294, 30)

In [26]:
# We'll go ahead and drop the rows that are missing UNIT_TYPE, etc as well. We're only losing about 3% of our data
# Considering that these are all missing the same data, they likely have an explanation.
# This data isn't very easily imputed, etc for missing values as well(most common vehicle type? etc)
# dv_crash[dv_crash['UNIT_TYPE'].isna()]
dv_crash.dropna(subset=['UNIT_TYPE'], inplace=True)
dv_crash.isna().sum()

DRIVERS_LICENSE_CLASS      388727
SEX                             0
AGE                        291805
SAFETY_EQUIPMENT                0
UNIT_TYPE                       0
NUM_PASSENGERS                  0
MAKE                            0
MODEL                           0
VEHICLE_DEFECT                  0
VEHICLE_TYPE                    0
VEHICLE_USE                     0
MANEUVER                        0
FIRST_CONTACT_POINT             0
POSTED_SPEED_LIMIT              0
WEATHER_CONDITION               0
LIGHTING_CONDITION              0
FIRST_CRASH_TYPE                0
TRAFFICWAY_TYPE                 0
ALIGNMENT                       0
ROADWAY_SURFACE_COND            0
ROAD_DEFECT                     0
CRASH_TYPE                      0
PRIM_CONTRIBUTORY_CAUSE         0
SEC_CONTRIBUTORY_CAUSE          0
STREET_DIRECTION                0
BEAT_OF_OCCURRENCE              0
CRASH_HOUR                      0
CRASH_DAY_OF_WEEK               0
CRASH_MONTH                     0
INJURY_LEVEL  

Now we have a 'clean' dataset that achieves our goals. We'll pickle this DataFrame and re-examine our FSM model next.

In [27]:
dv_crash.to_pickle("./Data/clean_joined_df.pkl")